In [1]:
def txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        text_list = [i for i in text]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text_list))]
        for i in label:
            entity = i.split('\t')
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        while len(text_list) > 32:            
            x.append(text_list[:32])
            y.append(label_list[:32])
            text_list = text_list[32:]
            label_list = label_list[32:]
        x.append(text_list)
        y.append(label_list)
    return [x, y]

In [2]:
x1, y1 = txt_to_list('../datasets/stage1/SampleData_deid.txt')

In [28]:
x1[0]

['醫',
 '師',
 '：',
 '你',
 '有',
 '做',
 '超',
 '音',
 '波',
 '嘛',
 '，',
 '那',
 '我',
 '們',
 '來',
 '看',
 '報',
 '告',
 '，',
 '有',
 '些',
 '部',
 '分',
 '有',
 '紅',
 '字',
 '耶',
 '。',
 '民',
 '眾',
 '：',
 '紅']

In [3]:
x2, y2 = txt_to_list('../datasets/stage2/train_1_update.txt')

In [4]:
x4, y4 = txt_to_list('../datasets/stage4/train_2.txt')

In [5]:
x = x1 + x2 + x4
len(x)

22532

In [6]:
y = y1 + y2 + y4
len(y)

22532

In [7]:
# print(x[0])

In [8]:
# print(y[0])

In [9]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

14420 14420
3605 3605
4507 4507


In [10]:
import kashgari
kashgari.config.use_cudnn_cell = True
from kashgari.tasks.labeling.models import BiLSTM_Model

model = BiLSTM_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10,
         )
model.evaluate(test_x, test_y)
model.save('model/BiLSTM')

╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 32, 100)           173600    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 32, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 32, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 32, 27)            6939      
_________________________________________________________________
activation (Activation)      (None, 32, 27)            0         
Total params: 416,059
Trainable params: 416,059
Non-trainable params: 0
_____________________________________________________

In [11]:
# loaded_model = kashgari.utils.load_model('model/BiLSTM')
# loaded_model.predict(test_x)[0]

In [12]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=32,
                          )
model = BiLSTM_Model(bert_embed)
model = BiLSTM_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10,
         )
model.evaluate(test_x, test_y)
model.save('model/BERT_BiLSTM')

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 32, 100)           173600    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 32, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 32, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 32, 27)            6939      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 27)            0         
Total params: 416,059
Trainable params: 416,059
Non-trainable params: 0
_____________________________________________________

In [26]:
loaded_model = kashgari.utils.load_model('model/BERT_BiLSTM')
# loaded_model.predict(test_x)

In [109]:
import pandas as pd
df = pd.read_csv('../datasets/stage4/development_2.csv')

In [110]:
df

,Unnamed: 0,article_id,text
0,0,0,醫師：這個要電腦看才有辦法，這是嚴重或一般的。民眾：這是沒關係啦。醫師：沒關係齁。民眾：他一...
1,1,1,醫師：就照舊這樣子。阿這一定要跟你講一下。家屬：沒關係啊。醫師：阿最近好嗎？民眾：都正常。醫...
2,2,2,醫師：就照這樣，我們就照這樣問診這樣而已啦，阿最近我們8月17號有抽血。家屬：對上上個禮拜。...
3,3,3,醫師：最近有怎麼樣嗎？民眾：都一樣欸。醫師：會痛還是怎麼樣嗎？民眾：都一樣。醫師：有發燒或是...
4,4,4,民眾：前一次是6月14號。醫師：嗯，14。民眾：13還是14。醫師：我看一下14啊，13和1...
...,...,...,...
65,65,65,個管師：來坐這裡。民眾：好，坐這裡嗎？好，靠你那麼近？個管師：靠我近一點好啊，不好嗎？會讓你...
66,66,66,個管師：好喘……哈哈哈哈。民眾：哈哈。個管師：好。小明這個月還好嗎？民眾：很好啊。個管師：很...
67,67,67,個管師：所以你是去篩檢的時候……民眾：恩。個管師：你去篩檢的時候然後許小姐跟你說你可以來這邊...
68,68,68,個管師：好喔，過去這一個月還好嗎？民眾：恩，還可以。個管師：那今天是請假？民眾：沒有，晚上上...


In [111]:
def predict_NER(model, text, sequence_length):
    x_list = list()
    while len(text) > sequence_length:
        x_list.append([i for i in text[:sequence_length]])
        text = text[sequence_length:]        
    x_list.append([i for i in text[:sequence_length]])
    y_list = model.predict(x_list)
    merge_list = list()
    for i in y_list:
        for j in i:
            merge_list.append(j)
    return merge_list   

In [112]:
def output_NER(article_id, text, y):
    output = str()
    flag = False
    for i, j in enumerate(y):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            output += '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
    return output

In [115]:
output = str()
for article_id, text in zip(df['article_id'], df['text']):
    y = predict_NER(model, text, 32)
    output += output_NER(article_id, text, y)    

In [116]:
print(output)

0	273	275	李醫	name
0	320	322	25	med_exam
0	326	328	25	med_exam
0	615	619	11點多	time
0	1168	1172	7月21	time
0	1571	1572	1	med_exam
0	1697	1699	第二	time
0	1707	1709	第二	time
0	2200	2203	13號	time
0	2214	2215	1	time
0	2226	2228	3天	time
0	2319	2321	禮拜	time
0	2332	2334	禮拜	time
0	2738	2740	林區	location
0	2807	2810	下禮拜	time
0	3306	3308	3天	time
0	3340	3343	下禮拜	time
1	59	62	108	med_exam
1	66	69	108	med_exam
1	457	460	五月中	time
2	27	32	8月17號	time
2	40	45	上上個禮拜	time
2	53	56	110	med_exam
2	85	88	110	med_exam
2	111	114	6.0	med_exam
2	253	256	126	time
2	279	281	2年	time
2	496	498	泰國	location
2	524	526	釣蝦	name
2	1114	1116	半年	time
2	1118	1120	半年	time
2	1147	1149	三個	time
2	1227	1231	兩個禮拜	time
3	183	185	7點	time
3	233	236	9點多	time
3	250	255	6月15號	time
3	256	260	16號左	time
3	265	268	16號	time
3	272	274	14	time
3	278	281	16號	time
3	302	306	1個禮拜	time
3	576	580	6月4號	time
3	582	584	前年	time
3	592	594	前年	time
3	1431	1433	禮拜	time
3	1443	1446	禮拜2	time
3	1456	1459	禮拜2	time
3	1489	1492	禮拜1	time
3	1873	1874	4	time
3	1875	1877	

In [ ]:
output_path='bert32.tsv'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output)